In [1]:
import dash
from dash import dcc
from dash import html
import plotly.express as px
import pandas as pd
import requests
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from datetime import datetime, timedelta

# Trending coins

In [2]:
trending_coins_url = 'https://api.coingecko.com/api/v3/search/trending'
trending_coins_info = requests.get(trending_coins_url).json()

In [3]:
print('Trending coins:')
for coin in trending_coins_info['coins']:
    try:
        url = f'''https://api.coingecko.com/api/v3/coins/{coin['item']['id']}?localization=en&tickers=false&market_data=true&community_data=false&developer_data=false&sparkline=false'''
        data = requests.get(url).json()
        print(f'''coin: {coin['item']['name']}, ticker: {coin['item']['symbol']}, market cap rank: {data['market_cap_rank']}, current price: {data['market_data']['current_price']['usd']}, perc price change in 24h: {round(data['market_data']['price_change_percentage_24h'], 2)}, perc market cap change in 24h: {round(data['market_data']['market_cap_change_percentage_24h_in_currency']['usd'], 2)}''')
        
    except KeyError:
        continue

Trending coins:
coin: Terra, ticker: LUNA, market cap rank: 61, current price: 0.00017257, perc price change in 24h: -9.16, perc market cap change in 24h: -7.29
coin: Unizen, ticker: ZCX, market cap rank: 601, current price: 0.733241, perc price change in 24h: 32.57, perc market cap change in 24h: 36.86
coin: AssetMantle, ticker: MNTL, market cap rank: 702, current price: 0.173812, perc price change in 24h: 40.44, perc market cap change in 24h: 41.11
coin: TerraUSD, ticker: UST, market cap rank: 59, current price: 0.103214, perc price change in 24h: 15.66, perc market cap change in 24h: 15.65
coin: Evmos, ticker: EVMOS, market cap rank: 170, current price: 1.17, perc price change in 24h: -35.72, perc market cap change in 24h: -32.65
coin: Flow, ticker: FLOW, market cap rank: 28, current price: 3.02, perc price change in 24h: 3.31, perc market cap change in 24h: 3.36
coin: STEPN, ticker: GMT, market cap rank: 73, current price: 1.42, perc price change in 24h: -5.79, perc market cap chan

In [4]:
#add mini plots for each trending coin with previous price (green or red, depending whether the price has risen in the last 7 days)
#-- ^ done --
#with 2-day prediction, printed in gray color
#add 'look up your coin' function, where it would return data on the coin similiar to the one presented in trending coins section
#with price data, market cap data, previous price and predictions
#maybe add crypto thumbnails for each trending cc?
#possibly: integration with Binance or other cc market for historical price data
# -- ^ WIP, currently FTX is supported, Binance and OKX coming in the near future
#rewrite notebook to .py script

# Global Crypto Data

In [5]:
gcd_url = 'https://api.coingecko.com/api/v3/global'
gcd = requests.get(gcd_url).json()['data']

In [6]:
#updated_at -> send cryptocurrency data to db with the timestamp
#updates every 0.25hr

In [7]:
pd.DataFrame({'timestamp': [pd.to_datetime(gcd['updated_at'], unit='s')], 'total_volume_usd': [gcd['total_volume']['usd']], 'active_cryptocurrencies': [gcd['active_cryptocurrencies']], 'market_cap_usd': [gcd['total_market_cap']['usd']],
             'btc_market_cap_perc': [gcd['market_cap_percentage']['btc']], 'eth_market_cap_perc': [gcd['market_cap_percentage']['eth']],
             'alts_market_cap_perc': [100-gcd['market_cap_percentage']['btc']-gcd['market_cap_percentage']['eth']]})

,timestamp,total_volume_usd,active_cryptocurrencies,market_cap_usd,btc_market_cap_perc,eth_market_cap_perc,alts_market_cap_perc
0,2022-05-18 19:54:38,8.716275e+10,13422,1.314276e+12,42.453488,18.219156,39.327356


In [8]:
#add total_volume_usd, market_cap_usd mini plots + short term predictions

# Retrieving crypto market data for trending coins 

In [9]:
supported_exchanges = ['FTX', 'Binance', 'OKX']

In [10]:
temp_for_df = []
for coin in trending_coins_info['coins']:
    url = f'''https://api.coingecko.com/api/v3/coins/{coin['item']['id']}/tickers'''
    r = requests.get(url).json()
    tmp = []
    for ticker in r['tickers']:
        if (ticker['market']['name'] in supported_exchanges) and (ticker['is_stale'] == False):
            tmp.append([ticker['market']['name'], ticker['base'], ticker['target'], ticker['volume']])
    try:
        temp_for_df.append(pd.DataFrame(tmp, columns=['exchange', 'base', 'quote', 'volume']).sort_values(by='volume', ascending=False).head(1).values[0])
    except IndexError:
        continue

In [11]:
where_to_look_for_interesting_coins = pd.DataFrame(temp_for_df, columns=['exchange', 'base', 'quote', 'volume'])

In [12]:
where_to_look_for_interesting_coins

,exchange,base,quote,volume
0,Binance,LUNA,BUSD,3.000977e+12
1,Binance,UST,BUSD,1.946800e+09
2,Binance,FLOW,USDT,7.243766e+06
3,Binance,GMT,USDT,3.343054e+08


In [13]:
class getExchangeData(ABC):
    
    @abstractmethod
    def get_ticker_info(self):
        pass
    
    @abstractmethod
    def get_ticker_info_url(self):
        pass

    @abstractmethod
    def get_start_time(self):
        pass
    
    @abstractmethod
    def get_interval(self):
        pass
    
    @abstractmethod
    def get_column_mapping(self):
        pass

In [14]:
class getFTXData(getExchangeData):
    
    def get_ticker_info(self, base, quote, interval, days_back):
        df = pd.DataFrame(requests.get(self.get_ticker_info_url(base, quote, interval, days_back)).json()['result'])
        df['startTime'] = pd.to_datetime(df.startTime)
        return df.rename(columns=self.get_column_mapping())
    
    def get_ticker_info_url(self, base, quote, interval, days_back):
        return f'https://ftx.com/api/markets/{base}/{quote}/candles?resolution={self.get_interval(interval)}&start_time={self.get_start_time(days_back)}'
    
    def get_start_time(self, days_back=0, minutes_back=0):
        return str((datetime.now()-timedelta(days=days_back, minutes=minutes_back)).timestamp()).split('.')[0]    
    
    def get_interval(self, interval):
        if interval[-1:] == 's':
            return int(interval[:-1])
        elif interval[-1:] == 'm':
            return int(interval[:-1])*60
        elif interval[-1:] == 'h':
            return int(interval[:-1])*3600
    
    def get_column_mapping(self):
        return {"startTime": "date_time", "time": "timestamp"}

In [15]:
def get_price_data_for_ticker(exch, base, quote):
        if exch == 'FTX':
            return getFTXData().get_ticker_info(base, quote, '1h', 7)
        elif exch == 'Binance':
            #getBinanceData().get_ticker_info(base, quote, '1h', 7)
            print(f'getting ticker data for {exch} {base}{quote}')
        elif exch == 'OKX':        
            #getOKXData().get_ticker_info(base, quote, '1h', 7)
            print(f'getting ticker data for {exch} {base}{quote}')

In [16]:
for row in where_to_look_for_interesting_coins.itertuples():
    df = get_price_data_for_ticker(row.exchange, row.base, row.quote)
    try:
        plt.plot(df.date_time, df.open, color=('green' if df.iloc[-1, 2] > df.iloc[0, 2] else 'red'), alpha=0.5)
        plt.xticks(rotation=45)
        plt.title(f'{row.exchange} {row.base}{row.quote}')
    except AttributeError:
        #skips if no data is available
        continue

getting ticker data for Binance LUNABUSD
getting ticker data for Binance USTBUSD
getting ticker data for Binance FLOWUSDT
getting ticker data for Binance GMTUSDT
